In [1]:
import wandb
import credentials as cre
import os 

wandb.login(key=cre.wandb_key)
run = wandb.init()
artifact = run.use_artifact('dylanon/text-summarization-model/slices:latest', type='dataset')
artifact_dir = artifact.download(root="datasets")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dylanonwic (dylanon). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\dylan/.netrc


In [2]:
from transformers import AutoTokenizer
import re
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", legacy=False)

c:\Users\dylan\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def preprocess_function(dataset):
    
    text_token_len = 512  #512
    sum_token_len = 84   #84
    # Convert the text and summary columns to tensors.
    input_text = dataset["text"]
    target_text = dataset["sum"]

    # this line just follow model card in huggingface
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
    input_text = [WHITESPACE_HANDLER(text) for text in input_text]
    
    # Tokenize the input and target text.
    text_token = tokenizer(input_text, truncation=True, padding=True, max_length=text_token_len)
    sum_token = tokenizer(target_text, truncation=True, padding=True, max_length=sum_token_len)

    # model input
    model_inputs = text_token
    model_inputs["labels"] = sum_token["input_ids"]
    return model_inputs

In [9]:
# Transform datasets
from datasets import load_dataset
import datasets

artifact_dir = os.path.abspath("datasets/slice_*.parquet")

# read artifact
dataset = load_dataset("parquet", data_files=artifact_dir, num_proc=16)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/dylan/.cache/huggingface/datasets/parquet/default-bd23060d2cb5baae/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [66]:
# select some data set.
'''
source,index_min,index_max
gigaword,0,3995558
thaisum,3995559,4376426
xlsum,8811187,9149046
'''
selected_ind = list(range(4376426)) + list(range(8811187,9149046))
tmp_dataset = dataset.sort('id')
tmp_dataset = tmp_dataset['train'].select(selected_ind)
dataset = tmp_dataset.shuffle(seed=1150)

Loading cached shuffled indices for dataset at C:\Users\dylan\.cache\huggingface\datasets\parquet\default-bd23060d2cb5baae\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-6cfe5e335dac4e51.arrow


In [67]:
dataset.save_to_disk('./datasets/datasets')

Saving the dataset (0/13 shards):   0%|          | 0/4714285 [00:00<?, ? examples/s]

In [82]:
# @title ##Prepare data
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['text','sum'])

Map:   0%|          | 0/4714285 [00:00<?, ? examples/s]

In [83]:
tokenized_dataset.save_to_disk('./datasets/tokenized_dataset')

Saving the dataset (0/31 shards):   0%|          | 0/4714285 [00:00<?, ? examples/s]

In [88]:
art = wandb.Artifact(f"tokenized-dataset", type="dataset")
local_path = os.path.abspath("./datasets/tokenized_dataset")

for file in os.listdir(local_path):
    path = os.path.join(local_path, file)
    try:
        if os.path.isdir(path):
            art.add_dir(path)
        else:
            art.add_file(path)
    except Exception as e: 
        print(e)

wandb.log_artifact(art)

<Artifact tokenized-dataset>

In [90]:
art = wandb.Artifact(f"datasets", type="dataset")
local_path = os.path.abspath("./datasets/datasets")

for file in os.listdir(local_path):
    path = os.path.join(local_path, file)
    try:
        if os.path.isdir(path):
            art.add_dir(path)
        else:
            art.add_file(path)
    except Exception as e: 
        print(e)

wandb.log_artifact(art)

<Artifact datasets>